In [1]:
import numpy as np

In [2]:
np.random.seed(0)

In [3]:
data = "hello world"
chars = list(set(data))
vocab_size = len(chars)

chars_to_indices = {char: i for i, char in enumerate(chars)}
indices_to_chars = {i: char for i, char in enumerate(chars)}

chars_to_indices, indices_to_chars

({'e': 0, 'd': 1, 'r': 2, 'l': 3, 'h': 4, 'w': 5, ' ': 6, 'o': 7},
 {0: 'e', 1: 'd', 2: 'r', 3: 'l', 4: 'h', 5: 'w', 6: ' ', 7: 'o'})

In [4]:
# RNN configs
embedding_size = vocab_size
hidden_size = 64
output_size = vocab_size

In [5]:
# Matrices
Wxh = np.random.randn(hidden_size, embedding_size)
Whh = np.random.randn(hidden_size, hidden_size)
bh = np.random.randn(1, hidden_size)

Why = np.random.randn(output_size, hidden_size)
by = np.random.randn(1, output_size)

In [6]:
def softmax(z):
    e_z = np.exp(z)
    return e_z / e_z.sum(axis=1)

def rnn(x_t, h_prev):
    assert x_t.shape == (1, embedding_size)
    assert h_prev.shape == (1, hidden_size)

    h_next = np.tanh(x_t @ Wxh.T + h_prev @ Whh.T + bh)
    probs = softmax(h_next @ Why.T + by)
    
    return probs, h_next

def sample(char, n):
    # sample rnn n times starting with the first char
    x = np.zeros((1, vocab_size))
    idx = chars_to_indices[char]
    x[:, idx] = 1 # one hot encoding
    h =  np.zeros((1, hidden_size)) # hidden state
    idxes = []
    
    for i in range(n):
        probs, h = rnn(x, h) # xt, hprev -> rnn -> probs, hnext
        idx = np.random.choice(vocab_size, p=probs.ravel()) # .ravel returns a 1d array
        
        x = np.zeros((1, vocab_size))
        x[:, idx] = 1
        idxes.append(idx)

    chars = "".join([indices_to_chars[i] for i in idxes])
    return chars

In [7]:
x_t = np.random.randn(1, embedding_size)
h_prev = np.random.randn(1, hidden_size)

probs, h_next = rnn(x_t, h_prev)
probs.shape, h_next.shape

((1, 8), (1, 64))

In [8]:
inp = [chars_to_indices[c] for c in data[:-1]]
target = [chars_to_indices[c] for c in data[1:]]

inp, target

([4, 0, 3, 3, 7, 6, 5, 7, 2, 3], [0, 3, 3, 7, 6, 5, 7, 2, 3, 1])

In [9]:
sample("h", 4)

'oww '

In [23]:
lr = 1e-2
for e in range(100):
    # compute loss
    loss = 0
    xs, ps, hs = {}, {}, {}
    h = np.zeros((1, hidden_size))
    for t in range(len(inp)):
        idx = inp[t]
        x = np.zeros((1, vocab_size))
        x[:, idx] = 1
        probs, h = rnn(x, h)
        xs[t] = x
        ps[t] = probs # save probs (we'll use this in backprop)
        hs[t] = h

        # cross entropy
        pred = probs[0, target[t]]
        loss += -np.log(pred)

    # backprop, calculate gradients
    dL_dWxh = np.zeros_like(Wxh)
    dL_dWhh = np.zeros_like(Whh)
    dL_dbh = np.zeros_like(bh)
    dL_Why = np.zeros_like(Why)
    dL_dby = np.zeros_like(by)
    dF_dh = np.zeros((1, hidden_size))

    for t in reversed(range(len(inp))):
        dL_dz2 = np.copy(ps[t])
        target_idx = target[t]
        dL_dz2[:, target_idx] -= 1

        # print(dL_dz2.shape)

        a = hs[t] * dL_dz2.T
        # print(a.shape)

        # 2nd layer
        dL_Why += hs[t] * dL_dz2.T
        dL_dby += dL_dz2


        # 1st layer
        dh_dz1 = 1 - hs[t] ** 2
        dL_dh = dF_dh + (dL_dz2 @ Why) 

        # print((xs[t] * dh_dz1.T * dL_dh.T).shape)

        dL_dWxh += xs[t] * dh_dz1.T * dL_dh.T
        dL_dWhh +=  hs[t] * dh_dz1.T * dL_dh.T
        dL_dbh +=  dh_dz1 * dL_dh

        # raise RuntimeError

        # dF_dh = dL_dh * (dh_dz1 @ Whh)

    # clip gradients
    for gradient in [dL_dWxh, dL_dWhh, dL_Why, dL_dbh, dL_dby]:
        np.clip(gradient, -1, 1, out=gradient)
        # np.clip(gradient, -5, 5, out=gradient)

    # gradient descent
    for weights, gradient in zip([Wxh, Whh, Why, bh, by], [dL_dWxh, dL_dWhh, dL_Why, dL_dbh, dL_dby]):
        weights -= lr * gradient

    # break
    
    print(f"[Loss]: {loss}")

[Loss]: 0.28650421268269377
[Loss]: 0.2856201842461065
[Loss]: 0.28472405238797843
[Loss]: 0.2838156306411589
[Loss]: 0.2828948011771858
[Loss]: 0.2819615160610996
[Loss]: 0.2810157971762303
[Loss]: 0.2800577349147106
[Loss]: 0.27908748577040754
[Loss]: 0.27810526898843296
[Loss]: 0.27711136242376055
[Loss]: 0.276106097748638
[Loss]: 0.27508985513067497
[Loss]: 0.27406305748601545
[Loss]: 0.273026164398071
[Loss]: 0.2719796657834399
[Loss]: 0.2709240753829382
[Loss]: 0.26985992415592697
[Loss]: 0.26878775365900415
[Loss]: 0.2677081094937502
[Loss]: 0.2666215349108297
[Loss]: 0.26552856465865415
[Loss]: 0.2644297191625696
[Loss]: 0.26332549911614644
[Loss]: 0.2622163805589683
[Loss]: 0.2611028105067171
[Loss]: 0.259985203189687
[Loss]: 0.25886393694576904
[Loss]: 0.25773935180410085
[Loss]: 0.2566117477858618
[Loss]: 0.25548138393985154
[Loss]: 0.25434847812216904
[Loss]: 0.25321320752180176
[Loss]: 0.25207570992750916
[Loss]: 0.2509360857257021
[Loss]: 0.2497944006146746
[Loss]: 0.2486

In [16]:
sample("h", 10)

'ello world'